# Validation


## Overview


The goal of the Niche Validation is to compare the potential vegetation types as
determined by Niche Vlaanderen with the actual presence of vegetation types, e.g.
as mapped in a vegetation map such as the Biological Valuation Map - [Biologische waarderingskaart (BWK)]( https://www.vlaanderen.be/inbo/de-biologische-waarderingskaart/wat-is-de-biologische-waarderingskaart/)

The vegetation map should be a spatial vector file (e.g. a shapefile) with:

- up to 9 habitat types in fields named HAB1-HAB9,
- the proportion of the shape they occupy in fields named pHAB1-pHAB9 (0-100 %). 
 
*Note that the field names are not case sensitive (hab1, phab1 can be used without any conversion) and HAB2 until HAB9/pHAB2 until pHAB9 are facultative*.

The habitat types in the vegetation map should be described using the official abbreviations of the Natura 2000 habitats and/or the regional important biotopes. Since those are different from the vegetation types in Niche, we provide a [mapping
table](https://github.com/inbo/niche_vlaanderen/blob/master/niche_vlaanderen/system_tables/hab_niche_join.csv) to link both vegetation typologies.

In the next steps, a (fictive) **niche model** is **created**. 

In [1]:
from niche_vlaanderen import Niche
nv = Niche()
nv.run_config_file("../tests/data/bwk_tiny/tiny.yaml")

ModuleNotFoundError: No module named 'niche_vlaanderen'

Next a **validation object** is **created** which contains the vegetation map with extra columns NICH_x_y for the corresponding niche vegetation types (x refers to the xth habitat, HABx). Note that some vegetation types of the Biological Valuation Map used in this example cannot be mapped to niche vegetation types (e.g. `gh` = 'geen habitat'/'no habitat' in the example below).

The `zonal_statistics` module is used in the background to calculate the potential vegetation for every shape.
Note the progress bar: in this example it is going fast, but for larger areas, calculation may take up to 10 minutes.

In [ ]:
from niche_vlaanderen.validation import NicheValidation
validation = NicheValidation(niche=nv, map="../tests/data/bwk_tiny/bwk_clip.shp")

It is possible to check the **mapping per vegetation** type by checking the `map` attribute of the validation object:

In [ ]:
validation.map

Let us plot the vegetation map together with the niche predictions for niche vegetation type 5 that will be used further as an example.

In [ ]:
mymap = validation.map.boundary.plot(ax=nv.plot(5)) 
validation.map.apply(lambda x: mymap.annotate(text=str(x['pHAB1']) + "% " + str(x['HAB1']) + "\n" + 
                                                     str(x['pHAB2']) + "% " + str(x['HAB2']) + "\n" + 
                                                     str(round(x['area_shape'], 3)) + " ha", 
                                                 xy=x.geometry.centroid.coords[0], 
                                                 ha='center',
                                                    color = "white"), 
                           axis=1);

A **summarized table** containing scores of the validation for the whole study area can be found by calling the `summary` method.

In [ ]:
validation.summary

with:

- area_effective: total area in ha with a given vegetation type, taking into account the proportion of the vegetation within each shape (pHAB)
- nonpot: total area with actual presence according to the vegetation map, but without potential presence according to niche
- nonpot_opt: total area with actual presence according to the vegetation map, but without potential presence according to niche (taking pHAB into account)
- pot: total area with actual presence according to the vegetation map and with potential presence according to niche
- polygon count: number of shapes with the actual vegetation type according to the vegetation map. If there are few polygons, the final score should be taken with caution.
- score: proportion of the total area with potential presence according to niche, compared to the total area of polygons with presence of a vegetation according to the vegetation map
- scoreopt: proportion of the total area with potential presence according to niche, compared to the actual area of a vegetation type according to the vegetation map (i.e. taking pHAB into account)


**Intermediate tables** giving values for each shape can be retrieved by their names:

* veg_present: table showing whether the vegetation type is present in this polygon according to the vegetation map (1 or 0)
* area_effective: area of the shape in ha taking into account the proportion of the vegetation = area polygon * pHAB
* area_nonpot: area with actual presence according to the vegetation map, but without potential presence according to niche
* area_nonpot_phab: area with actual presence according to the vegetation map, but without potential presence according to niche (taking pHAB into account)
  * calculated as: If([area_effective]>[area_pot];[area_effective]-[area_pot];0)
* area_pot: area with actual presence according to the vegetation map and with potential presence predicted by niche
* area_pot_perc: proportion (%) of the area with correct predicted area (not taking into account pHAB)
  * calculated as: 100*[area_pot]/ ([area_pot]+[area_nonpot])
* area_pot_perc_phab: proportion (%) of the area with correct predicted area (taking into account pHAB)
  * calculated as: If([area_pot]<[area_effective];100*[ area_pot]/([area_effective]);100)

In [ ]:
validation.veg_present

In [ ]:
validation.area_effective

In [ ]:
validation.area_nonpot

In [ ]:
validation.area_nonpot_phab

In [ ]:
validation.area_pot

In [ ]:
validation.area_pot_perc

In [ ]:
validation.area_pot_perc_phab

## Using another field of the vegetation map as ID

You can use any field of the vegetation map as ID instead of the standard shape_id.

In [ ]:
validation1 = NicheValidation(niche=nv, map="../tests/data/bwk_tiny/bwk_clip.shp", id = "myid")

All the results per polygon will show the selected field as identifier, e.g.:

In [ ]:
validation1.veg_present

## Using a custom vegetation table

When creating a validation, it is possible to supply a custom mapping table between the vegetation types in the map and niche types. This file must be a csv with two columns, HAB and NICHE, containing the vegetation type on the map in the HAB column, and the corresponding Niche vegetation type in the NICHE column. 

Thanks to this option you can use a vegetation map with another vegetation typology than the Natura 2000 habitats/regional important biotopes or modify the link between vegetation types on the vegetation map and Niche vegetation types.

In [ ]:
validation1 = NicheValidation(niche=nv, map="../tests/data/bwk_tiny/bwk_clip.shp", 
                              mapping_file="custom_vegetation.csv")

In [ ]:
validation1.mapping

## Setting the spatial resolution of the overlay

The actual overlay will compare the grid cell centres (niche predictions) with the shapefile (vegetation map). 
By default an upscaling factor of 5 (25 points per grid cell) is used to ensure sufficient coverage of the shapefile. For niche models of 20-25m resolution, this default value will lead to a spatial resolution of 1m, which is sufficient given the common scale of the vegetation map.

![comparison between 1 and 5 upscaling](_static/png/bwk_overlay.png)

A greater upscaling factor can be used if the polygons of the vegetation map are particularly small compared to the resolution of the Niche predictions.

If upscaling is set to one (not used), only the grid center will be used, which may lead to warnings because some shapes are not covered. 

In [ ]:
validation1 = NicheValidation(niche=nv, map="../tests/data/bwk_tiny/bwk_clip.shp", upscale=1)

## Storing the analysis

Finally, the whole project can be saved. This will save the summary tables and the intermediate tables in a folder, and create a geopackage (vector spatial file) which contains the habitat map with all intermediate tables joined to it.

In [ ]:
validation.write('_validation', overwrite_files=True)

The exported files are:

In [ ]:
import os
os.listdir('_validation')